In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


%matplotlib inline

### Reading in, filtering, and examining the heart attack cost disparities data

In [80]:
# your path to the data file may vary!

ha_costs = pd.read_csv('../data/mmd_heart_attack_data.csv') 
print(ha_costs.shape)
print(ha_costs.head(2))

(2656, 17)
   year geography             measure         adjustment      analysis  \
0  2017    County  Average total cost  Unsmoothed actual  Base measure   
1  2017    County  Average total cost  Unsmoothed actual  Base measure   

                       domain                    condition primary_sex  \
0  Primary chronic conditions  Acute myocardial infarction         All   
1  Primary chronic conditions  Acute myocardial infarction         All   

  primary_age     primary_dual  fips          county    state  urban  \
0         All  Dual & non-dual  1001  Autauga County  ALABAMA  Urban   
1         All  Dual & non-dual  1003  Baldwin County  ALABAMA  Rural   

  primary_race primary_denominator  analysis_value  
0          All           undefined           40470  
1          All           undefined           36615  


### Now getting the cancer data

In [81]:
cancer_costs = pd.read_csv('../data/mmd_cancer_data.csv')
print(cancer_costs.shape)
print(cancer_costs.head(2))

(3165, 17)
   year geography             measure         adjustment      analysis  \
0  2017    County  Average total cost  Unsmoothed actual  Base measure   
1  2017    County  Average total cost  Unsmoothed actual  Base measure   

                       domain                                   condition  \
0  Primary chronic conditions  Cancer, Colorectal, Breast, Prostate, Lung   
1  Primary chronic conditions  Cancer, Colorectal, Breast, Prostate, Lung   

  primary_sex primary_age     primary_dual  fips          county    state  \
0         All         All  Dual & non-dual  1001  Autauga County  ALABAMA   
1         All         All  Dual & non-dual  1003  Baldwin County  ALABAMA   

   urban primary_race primary_denominator  analysis_value  
0  Urban          All           undefined           19293  
1  Rural          All           undefined           17151  


### Getting the income data and cleaning it a bit

In [82]:
income = pd.read_csv('../data/irs_county_2016.csv')
income.head(2)

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,Alabama,1,26450,14680,9740,680,17780,...,4300,9256,70,57,0,0,2590,3685,11510,35079
1,1,AL,0,Alabama,2,285760,217880,25170,39740,143390,...,70050,40569,0,0,0,0,22720,11109,237630,263152


In [83]:
income = income[['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']]
income.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']
income.head(2)

,state,county,income_bucket,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
0,AL,Alabama,1,26450,14680,9740,680,40700,5590,13000,-1679314,19140,-1657452,60,203
1,AL,Alabama,2,285760,217880,25170,39740,296830,78450,48270,1582247,285760,1632624,4180,10772


### Week two coding tasks
#### Replacing coded values in the `income_bucket` column with descriptive text
- create a dictionary mapping codes to descriptions
- use `replace()` to update the df with text

In [84]:
income_dict = {0:'Total', 1: 'Under $1', 2: 'Between 1 and $10,000', 3: 'Between 10,000 and $25,000',
              4: 'Between 25,000 and $50,000', 5: 'Between 50,000 and $75,000', 
               6: 'Between 75,000 and $100,000', 7: 'Between 100,000 and $200,000', 
               8:'$200,000 or more'}

In [85]:
income.income_bucket = income.income_bucket.replace(income_dict)
income.head(2)

,state,county,income_bucket,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
0,AL,Alabama,Under $1,26450,14680,9740,680,40700,5590,13000,-1679314,19140,-1657452,60,203
1,AL,Alabama,"Between 1 and $10,000",285760,217880,25170,39740,296830,78450,48270,1582247,285760,1632624,4180,10772


#### Creating a new df that aggregates by county to get the totals for each county

In [86]:
income_county_agg = income.groupby(['state','county']).agg('sum').reset_index()
income_county_agg.head(2)

,state,county,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp
0,AK,Alaska,348070,175480,126660,37340,654950,205660,71410,23514361,347600,23858011,83270,167460
1,AK,Aleutians East Borou,890,480,230,150,1570,500,160,42834,890,43596,190,466


In [87]:
income_county_agg['avg_income'] = round(income_county_agg.total_inc_amt * 1000 / income_county_agg.returns_with_total_inc, 0)
income_county_agg.head(3)

,state,county,return_count,single_returns,joint_returns,head_of_house_returns,exemptions,dependents,elderly,agi,returns_with_total_inc,total_inc_amt,returns_with_unemployment,unemployment_comp,avg_income
0,AK,Alaska,348070,175480,126660,37340,654950,205660,71410,23514361,347600,23858011,83270,167460,68636.0
1,AK,Aleutians East Borou,890,480,230,150,1570,500,160,42834,890,43596,190,466,48984.0
2,AK,Aleutians West Censu,2110,1150,600,290,3860,1260,350,126905,2110,128069,480,1043,60696.0


### Create a merged DataFrame for Heart Attack Costs and Income, keeping just `county`, `urban`, `analysis_value`, and `avg_income`; then do the same for Cancer Costs

In [88]:
# we only need the county and the average income from income_county_agg
county_incomes = income_county_agg[['state','county', 'avg_income']]
county_incomes.head(2)

,state,county,avg_income
0,AK,Alaska,68636.0
1,AK,Aleutians East Borou,48984.0


In [89]:
# we only need state, county, urban, and analysis_value columns from the heart attack costs
ha_costs = ha_costs[['state','county', 'urban', 'analysis_value']]

In [90]:
ha_costs.head(2)

,state,county,urban,analysis_value
0,ALABAMA,Autauga County,Urban,40470
1,ALABAMA,Baldwin County,Rural,36615


#### Need to get the state columns to match, so let's use the state_abbrev data to map them

In [91]:
state_abbrev = pd.read_csv('../data/state_abbrev.csv')
state_abbrev.head(3)

,name,abbrev
0,ALABAMA,AL
1,ALASKA,AK
2,ARIZONA,AZ


In [92]:
ha_costs['state'] = ha_costs.state.map(state_abbrev.set_index('name')['abbrev'].to_dict())
ha_costs.head()

,state,county,urban,analysis_value
0,AL,Autauga County,Urban,40470
1,AL,Baldwin County,Rural,36615
2,AL,Barbour County,Rural,46509
3,AL,Bibb County,Urban,42949
4,AL,Blount County,Urban,50067


In [93]:
ha_costs2 = pd.merge(ha_costs, county_incomes, on= ['state','county'], how = 'inner')
ha_costs2.head(2)

,state,county,urban,analysis_value,avg_income
0,AL,Autauga County,Urban,40470,55843.0
1,AL,Baldwin County,Rural,36615,62832.0


In [94]:
ha_costs2['cost_income_ratio'] = ha_costs2.analysis_value / ha_costs2.avg_income
ha_costs2.describe()

,analysis_value,avg_income,cost_income_ratio
count,2619.000000,2619.000000,2619.000000
mean,49315.675830,53604.476900,0.970133
std,9536.867466,15408.818496,0.273106
min,21954.000000,30311.000000,0.145986
25%,43087.500000,44429.000000,0.792187
50%,48396.000000,50099.000000,0.944243
75%,54590.000000,58295.000000,1.121650
max,111757.000000,214890.000000,2.530495


In [95]:
# we only need county, urban, and analysis_value columns from the cancer costs
cancer_costs = cancer_costs[['state','county', 'urban', 'analysis_value']]

In [96]:
cancer_costs['state'] = cancer_costs.state.map(state_abbrev.set_index('name')['abbrev'].to_dict())
cancer_costs.head()

,state,county,urban,analysis_value
0,AL,Autauga County,Urban,19293
1,AL,Baldwin County,Rural,17151
2,AL,Barbour County,Rural,19469
3,AL,Bibb County,Urban,17234
4,AL,Blount County,Urban,20317


In [97]:
cancer_costs2 = pd.merge(cancer_costs, county_incomes, on= ['state','county'], how = 'left')
cancer_costs2.head(2)

,state,county,urban,analysis_value,avg_income
0,AL,Autauga County,Urban,19293,55843.0
1,AL,Baldwin County,Rural,17151,62832.0


In [98]:
cancer_costs2['cost_income_ratio'] = cancer_costs2.analysis_value / cancer_costs2.avg_income
cancer_costs2.describe()

,analysis_value,avg_income,cost_income_ratio
count,3165.000000,3066.000000,3066.000000
mean,20480.169984,52825.301696,0.414977
std,3832.423489,14973.846719,0.120398
min,2628.000000,21434.000000,0.065843
25%,18401.000000,43913.000000,0.335343
50%,20211.000000,49692.000000,0.407073
75%,22349.000000,57446.500000,0.480998
max,55920.000000,214890.000000,1.317388


### Week 5 Coding Tasks

#### logistic regression model for myocardial infarction costs

- create target column (1 for cost-income ratio above the mean 0 if at or below the mean)
- encode the urban column
- split train/test
- use urban column to predict

In [99]:
ha_costs2.cost_income_ratio

0       0.724710
1       0.582744
2       1.154958
3       0.906480
4       1.010740
          ...   
2614    0.865683
2615    0.850084
2616    0.145986
2617    1.203812
2618    1.212371
Name: cost_income_ratio, Length: 2619, dtype: float64

In [100]:
# create target variable
ha_cost_income_ratio_mean = ha_costs2.cost_income_ratio.mean()
ha_costs2['cost_ratio_above_mean'] = [1 if ratio > ha_cost_income_ratio_mean else 0 for ratio in ha_costs2.cost_income_ratio]

In [101]:
ha_costs2.cost_ratio_above_mean.value_counts(normalize = True)

0    0.539137
1    0.460863
Name: cost_ratio_above_mean, dtype: float64

In [102]:
ha_costs2.head(2)

,state,county,urban,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean
0,AL,Autauga County,Urban,40470,55843.0,0.724710,0
1,AL,Baldwin County,Rural,36615,62832.0,0.582744,0


In [103]:
# encode urban/rural
ha_costs2 = pd.get_dummies(ha_costs2, columns = ['urban'], drop_first = True)
ha_costs2.head(2)

,state,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban
0,AL,Autauga County,40470,55843.0,0.724710,0,1
1,AL,Baldwin County,36615,62832.0,0.582744,0,0


In [104]:
X = ha_costs2[['urban_Urban']]
y = ha_costs2.cost_ratio_above_mean

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)

In [106]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [107]:
y_pred = logistic_model.predict(X_test)

In [108]:
print(metrics.accuracy_score(y_test, y_pred))

0.6152671755725191


#### the naive model (predicting the cost-income ratio above the mean for all cases) would have accuracy .527473

In [109]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:    ', metrics.confusion_matrix(y_test, y_pred)[1])

                 Pred Below Mean:  Pred Above Mean:
    Actual Below Mean:     [178 180]
    Actual Above Mean:     [ 72 225]


In [110]:
y_pred_prob = logistic_model.predict_proba(X_test)[:,1]
print('Area Under Curve:', metrics.roc_auc_score(y_test, y_pred_prob))

Area Under Curve: 0.627391230743186


#### Let's add another predictor - the Health Factors z-score from the county health rankings: [Robert Wood Johnson Foundation](https://www.countyhealthrankings.org)

![health factors](../data/health_factors.png)

In [111]:
health_rankings = pd.read_excel('../data/2018 County Health Rankings Data - v2.xls', 
                                sheet_name = 'Outcomes & Factors Rankings',
                               header = [0,1])

In [112]:
health_rankings.head(2)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                FIPS              State             County   
0               1001            Alabama            Autauga   
1               1003            Alabama            Baldwin   

    Unnamed: 3_level_0 Health Outcomes          Health Factors           
  # of Ranked Counties            Rank Quartile           Rank Quartile  
0                   67              11        1              8        1  
1                   67               3        1              3        1

In [113]:
health_rankings.columns = ['fips', 'state', 'county', 'ranked_county_count', 'outcomes_rank', 'outcomes_quartile', 
                           'factors_rank', 'factors_quartile']
health_rankings.head()

,fips,state,county,ranked_county_count,outcomes_rank,outcomes_quartile,factors_rank,factors_quartile
0,1001,Alabama,Autauga,67,11,1,8,1
1,1003,Alabama,Baldwin,67,3,1,3,1
2,1005,Alabama,Barbour,67,34,2,56,4
3,1007,Alabama,Bibb,67,41,3,37,3
4,1009,Alabama,Blount,67,14,1,19,2


#### we need to drop the state entries (where county is `NaN`) and remove the word "County" from  `ha_costs2`in order to merge the datasets

In [115]:
health_rankings = health_rankings.dropna()

In [116]:
ha_costs2.county = ha_costs2.county.str[0:-7]

In [117]:
ha_costs2.head(2)

,state,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban
0,AL,Autauga,40470,55843.0,0.724710,0,1
1,AL,Baldwin,36615,62832.0,0.582744,0,0


#### Need to make the Health Rankings state an abbreviation 
- You'll need to make the state all upper case so you can use the lookup table

In [118]:
health_rankings['state'] = health_rankings.state.str.upper().map(state_abbrev.set_index('name')['abbrev'].to_dict())
health_rankings.head()

,fips,state,county,ranked_county_count,outcomes_rank,outcomes_quartile,factors_rank,factors_quartile
0,1001,AL,Autauga,67,11,1,8,1
1,1003,AL,Baldwin,67,3,1,3,1
2,1005,AL,Barbour,67,34,2,56,4
3,1007,AL,Bibb,67,41,3,37,3
4,1009,AL,Blount,67,14,1,19,2


In [134]:
health_factors = health_rankings[['state', 'county', 'factors_quartile']]

In [135]:
ha_with_health_factors =pd.merge(ha_costs2, health_factors, on = 'county', how = 'left')
ha_with_health_factors.head(2)

,state_x,county,analysis_value,avg_income,cost_income_ratio,cost_ratio_above_mean,urban_Urban,state_y,factors_quartile
0,AL,Autauga,40470,55843.0,0.724710,0,1,AL,1
1,AL,Baldwin,36615,62832.0,0.582744,0,0,AL,1


In [136]:
ha_with_health_factors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13215 entries, 0 to 13214
Data columns (total 9 columns):
state_x                  13215 non-null object
county                   13215 non-null object
analysis_value           13215 non-null int64
avg_income               13215 non-null float64
cost_income_ratio        13215 non-null float64
cost_ratio_above_mean    13215 non-null int64
urban_Urban              13215 non-null uint8
state_y                  13176 non-null object
factors_quartile         13176 non-null object
dtypes: float64(2), int64(2), object(4), uint8(1)
memory usage: 942.1+ KB


In [146]:
ha_with_health_factors.factors_quartile.unique()

array([1, 4, 3, 2, 'NR', nan, 5], dtype=object)

In [143]:
ha_with_health_factors.factors_quartile = ha_with_health_factors.factors_quartile.astype(int, errors='ignore')
ha_with_health_factors.shape

(13215, 9)

In [147]:
ha_with_health_factors = ha_with_health_factors.loc[ha_with_health_factors.factors_quartile.isin([1, 2, 3, 4, 5])]


In [148]:
X = ha_with_health_factors[['urban_Urban', 'factors_quartile']]
y = ha_with_health_factors.cost_ratio_above_mean


In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [151]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [152]:
y_pred = logistic_model.predict(X_test)

In [153]:
print(metrics.accuracy_score(y_test, y_pred))

0.6072738386308069


In [154]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:   ', metrics.confusion_matrix(y_test, y_pred)[1])

                 Pred Below Mean:  Pred Above Mean:
    Actual Below Mean:     [896 900]
    Actual Above Mean:    [ 385 1091]


In [155]:
pd.DataFrame({'variable': ['intercept'] + list(X.columns),
                             'coefficient': [logistic_model.intercept_] + list(logistic_model.coef_[0])})

,variable,coefficient
0,intercept,[-0.13185493182741734]
1,urban_Urban,-1.18991
2,factors_quartile,0.13703
